# Scraping _______ from _______

## 2.1 Contents<a id='2.1_Contents'></a>
* [2.1 Importing Relevant Tools](#2.1_Importing)
* [2.2 Defining the Request](#2.2_URL)
* [2.3 Grab | Date](#2.4_scrape_date)
* [2.4 Grab | Header](#2.4_scrape_header)
* [2.5 Grab | Content](#2.4_scrape_content)
* [2.6 Clean | Send to DataFrame](#2.5_review)
* [2.7 Save](#2.6_save)


**Importing Relevant Tools**<a id='2.1_Importing'></a>

In [157]:
import json
import pandas as pd
from bs4 import BeautifulSoup as soup
import requests
import csv
import re
import pickle

%reload_ext watermark

In [158]:
#the below needs to be reviewed for all websites; notably the time format

from datetime import date
from datetime import datetime
today = date.today()
d = today.strftime("%m-%d-%y")

For reference.

In [159]:
%watermark -d -t -v -p pandas

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.19.0

pandas: 1.1.3



**Defining the Request**<a id='2.2_URL'></a>

In [160]:
url = 'https://www.nytimes.com/2020/06/28/world/asia/china-hong-kong-national-security-law.html'.format(d)
url

'https://www.nytimes.com/2020/06/28/world/asia/china-hong-kong-national-security-law.html'

In [161]:
html = requests.get(url)

In [162]:
bsobj = soup(html.content,'lxml')
# bsobj

**Grab | Date**<a id='2.4_scrape_date'></a>

In [163]:
for date in bsobj.find('div',{'class':'css-1hdytw'}):
    print(date.text.strip())

Published June 28, 2020Updated June 30, 2020


Lot's of cleaning to do.

**Grab | Header**<a id='2.4_scrape_header'></a>

In [164]:
for title in bsobj.findAll("h1"):
    print(format(title.text))

What China’s New National Security Law Means for Hong Kong


Confirmed that's the title.

**Grab | Content**<a id='2.4_scrape_content'></a>

In [165]:
for news in bsobj.findAll('div',{'class':'css-1fanzo5 StoryBodyCompanionColumn'}):
    print(news.text.strip())

Chinese lawmakers have approved a national security law, drastically curbing political protest and dissent in Hong Kong and adding to Beijing’s tensions with Western powers.The National People’s Congress Standing Committee — an arm of China’s Communist Party-run legislature — discussed the draft law twice this month, including a three-day session that started on Sunday. But even as Hong Kong news media reported on Tuesday that the law had passed, the city’s residents were still waiting to see its full text.
Here is what the move means for Hong Kong.A law to curb opposition in Hong Kong.
China’s Communist Party leaders have long worried about opposition to their rule in Hong Kong, which was a British colony until 1997. The Basic Law, which enshrines Hong Kong’s special legal status, says the semiautonomous territory should enact legislation that outlaws “any act of treason, secession, sedition, subversion.”
Full details have not been released, but the security law is expected to deter s

Confirmed that's the bottom.

**Clean | Send to DataFrame**<a id='2.5_review'></a>

First the date from string to a datetime object.

In [166]:
for day in bsobj.find('div',{'class':'css-1hdytw'}):
    day_pub = day.text.strip()

In [167]:
day_pub = re.sub('Published ','',day_pub)
day_pub = re.sub('Updated June 30, 2020','',day_pub)
day_pub = re.sub(',','',day_pub)
day_pub = re.sub('June','06',day_pub)
day_pub = re.sub(' ','-',day_pub)
day_pub = datetime.strptime(day_pub, '%m-%d-%Y').date()
day_pub

datetime.date(2020, 6, 28)

In [168]:
df_date = pd.DataFrame([day_pub])

In [169]:
type(df_date)

pandas.core.frame.DataFrame

In [170]:
df_date

,0
0,2020-06-28


Now the title.

In [171]:
for title_s in bsobj.findAll("h1"):
    title_list = format(title_s.text)

In [172]:
df_title = pd.DataFrame([title_list])

In [173]:
df_title

,0
0,What China’s New National Security Law Means f...


In [174]:
type(df_title)

pandas.core.frame.DataFrame

These items are manually added.

In [176]:
country = 'US'
df_country = pd.DataFrame([country])
source = 'NY Times'
df_source = pd.DataFrame([source])
file_name = 'nytimes_5'
df_file_name = pd.DataFrame([file_name])


Finally, the news.

In [177]:
news1 = []
for news in bsobj.findAll('div',{'class':'css-53u6y8'}):
    news1.append(news.text.strip())

In [178]:
news1

['Chinese lawmakers have approved a national security law, drastically curbing political protest and dissent in Hong Kong and adding to Beijing’s tensions with Western powers.The National People’s Congress Standing Committee — an arm of China’s Communist Party-run legislature — discussed the draft law twice this month, including a three-day session that started on Sunday. But even as Hong Kong news media reported on Tuesday that the law had passed, the city’s residents were still waiting to see its full text.',
 'Here is what the move means for Hong Kong.A law to curb opposition in Hong Kong.',
 'China’s Communist Party leaders have long worried about opposition to their rule in Hong Kong, which was a British colony until 1997. The Basic Law, which enshrines Hong Kong’s special legal status, says the semiautonomous territory should enact legislation that outlaws “any act of treason, secession, sedition, subversion.”',
 'Full details have not been released, but the security law is expec

In [179]:
len(news1)

9

In [180]:
news1[8]

'Prime Minister Boris Johnson of Britain has promised to allow nearly three million people from Hong Kong to live and work in the country. Mr. Johnson, however, has left unanswered questions about how those admitted might be able to obtain British citizenship.Taiwan said this month that it would expand efforts to provide refuge to protesters and others who wish to leave Hong Kong. The government said it could, in certain cases, provide work and study visas, as well as assistance securing housing and jobs.Elaine Yu contributed reporting from Hong Kong.'

In [181]:
news1[8] = re.sub('Elaine Yu contributed reporting from Hong Kong.','',news1[8])

In [182]:
df_news = pd.DataFrame()

In [183]:
df_news['article_body'] = news1

In [184]:
df_news.head(2)

,article_body
0,Chinese lawmakers have approved a national sec...
1,Here is what the move means for Hong Kong.A la...


In [185]:
df_news['article_body'] = df_news.article_body.str.cat(sep=' ')

In [186]:
df_news = df_news.article_body[0]

In [187]:
df_news = df_news.replace(r'\\?','')

In [188]:
df_news = pd.DataFrame([df_news])

In [189]:
type(df_news)

pandas.core.frame.DataFrame

In [190]:
df_news.columns = ['Article']

In [191]:
df_news.head()

,Article
0,Chinese lawmakers have approved a national sec...


**Bringing it together.**<a id='2.5_bit'></a>

In [192]:
df_5_nytimes = pd.concat([df_file_name,df_date,df_source,df_country,df_title,df_news],axis = 1, ignore_index=False)

In [193]:
df_5_nytimes.columns = ['file_name','date','source','country','title','article']

In [194]:
df_5_nytimes.head()

,file_name,date,source,country,title,article
0,nytimes_5,2020-06-28,NY Times,US,What China’s New National Security Law Means f...,Chinese lawmakers have approved a national sec...


**Saving**<a id='2.6_save'></a>

In [195]:
cd

C:\Users\rands


Saving it to Excel.

In [196]:
# df = pd.DataFrame(reviewlist)

# index=False below so that we don't get the dataframe index on the side; we just use the excel index
df_5_nytimes.to_csv('./_Capstone_Two_NLP/data/_news/nytimes_5.csv', index=False)

print('Complete')

Complete
